In [1]:
import gc
import numpy as np
import pandas as pd
import polars as pl
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
from scipy.signal import find_peaks
from scipy import signal

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchinfo import summary
import os
import nni
from torchviz import make_dot
import graphviz

In [8]:
graphviz_path = r'C:\Program Files\Graphviz\bin'
os.environ['PATH'] += os.pathsep + graphviz_path

In [2]:
n_splits = 5
n_epochs = 20
batch_size = 16
features = ["log_anglez_std", 
            "log_enmo", 
            "valid_flag", 
            # "min_mod_15_plus_1",
            # "hour_sin",
            # "hour_cos"
            ]
filters = 24
initial_channels_number = len(features) * filters
seed = 1

In [3]:
class ConvBNReLU(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups=1):
        super().__init__()
        
        if stride == 1:
            padding = "same"
        else:
            padding = (kernel_size - stride) // 2
        self.layers = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=groups),
            nn.BatchNorm1d(out_channels),
            nn.ReLU()
        )
    
    def forward(self, x):
        x_out = self.layers(x)
        return x_out


class SEBlock(nn.Module):
    def __init__(self, n_channels, se_ratio):
        super().__init__()
        
        self.layers = nn.Sequential(
            nn.AdaptiveAvgPool1d(output_size=1),  #  Global Average Pooling
            nn.Conv1d(n_channels, n_channels//se_ratio, kernel_size=1),
            nn.ReLU(),
            nn.Conv1d(n_channels//se_ratio, n_channels, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x_out = torch.mul(x, self.layers(x))
        return x_out


class ResBlock(nn.Module):
    def __init__(self, n_channels, kernel_size, se_ratio):
        super().__init__()
        
        self.layers = nn.Sequential(
            ConvBNReLU(n_channels, n_channels, kernel_size, stride=1),
            ConvBNReLU(n_channels, n_channels, kernel_size, stride=1),
            SEBlock(n_channels, se_ratio)
        )
    
    def forward(self, x):
        x_re = self.layers(x)
        x_out = x + x_re
        return x_out
    

class UNet1d(nn.Module):
    def __init__(self, input_channels, initial_channels, initial_kernel_size,
                 down_channels, down_kernel_size, down_stride, res_depth, res_kernel_size, se_ratio, out_kernel_size):
        super().__init__()
        self.down_kernel_size = down_kernel_size
        self.down_stride = down_stride
        
        self.initial_layers = ConvBNReLU(input_channels, initial_channels, initial_kernel_size, stride=1, groups=input_channels)
        # self.initial_layers = nn.Sequential(
        #     ConvBNReLU(input_channels, initial_channels, initial_kernel_size, stride=1, groups=input_channels),
        #     ConvBNReLU(initial_channels, initial_channels, initial_kernel_size, stride=1, groups=1)
        # )
        
        self.down_layers = nn.ModuleList()
        for i in range(len(down_channels)):
            if i == 0:
                in_channels = initial_channels
            else:
                in_channels = down_channels[i-1] + input_channels
            out_channels = down_channels[i]
            kernel_size = down_kernel_size[i]
            stride = down_stride[i]
            
            block = []
            block.append(ConvBNReLU(in_channels, out_channels, kernel_size, stride))
            for j in range(res_depth):
                block.append(ResBlock(out_channels, res_kernel_size, se_ratio))
            self.down_layers.append(nn.Sequential(*block))
        
        self.up_layers = nn.ModuleList()
        for i in range(len(down_channels)-1, 0, -1):
            in_channels = out_channels + down_channels[i]
            out_channels = down_channels[i]
            kernel_size = down_kernel_size[i]
            self.up_layers.append(ConvBNReLU(in_channels, out_channels, kernel_size, stride=1))
        
        self.out_layers = nn.Conv1d(down_channels[1], 1, out_kernel_size, padding="same")
    
    def forward(self, x):
        outs = []
        x_avg = x
        x = self.initial_layers(x)
        
        for i in range(len(self.down_layers)):
            x_out = self.down_layers[i](x)
            if i == len(self.down_layers) - 1:
                x = x_out
            else:
                outs.append(x_out)
                kernel_size = self.down_kernel_size[i]
                stride = self.down_stride[i]
                padding = (kernel_size - stride) // 2
                x_avg = F.avg_pool1d(x_avg, kernel_size, stride, padding)
                x = torch.cat([x_out, x_avg], dim=1)
        
        for i in range(len(self.up_layers)):
            scale_factor = self.down_stride[-i-1]
            x = F.interpolate(x, scale_factor=scale_factor, mode="linear")
            x = torch.cat([x, outs[-i-1]], dim=1)
            x = self.up_layers[i](x)
        
        x_out = self.out_layers(x)
        x_out = x_out[:, 0, 180:-180]
        
        return x_out

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = UNet1d(
    input_channels=3,
    initial_channels=initial_channels_number, #should be adjust to multiply of feature amounts
    initial_kernel_size=15,
    down_channels=(initial_channels_number, initial_channels_number, initial_channels_number),
    down_kernel_size=(12, 15, 15),
    down_stride=(12, 9, 5),  # first element must be 12
    res_depth=3,
    res_kernel_size=15,
    se_ratio=4,
    out_kernel_size=21,
)

In [5]:
print(summary(
                model=model,
                input_size=(batch_size, 3, 21600),
                col_names=["input_size", "output_size", "num_params"],
                col_width=20
            ))

Layer (type:depth-idx)                                       Input Shape          Output Shape         Param #
UNet1d                                                       [16, 3, 21600]       [16, 1440]           --
├─ConvBNReLU: 1-1                                            [16, 3, 21600]       [16, 72, 21600]      --
│    └─Sequential: 2-1                                       [16, 3, 21600]       [16, 72, 21600]      --
│    │    └─Conv1d: 3-1                                      [16, 3, 21600]       [16, 72, 21600]      1,152
│    │    └─BatchNorm1d: 3-2                                 [16, 72, 21600]      [16, 72, 21600]      144
│    │    └─ReLU: 3-3                                        [16, 72, 21600]      [16, 72, 21600]      --
├─ModuleList: 1-2                                            --                   --                   --
│    └─Sequential: 2-2                                       [16, 72, 21600]      [16, 72, 1800]       --
│    │    └─ConvBNReLU: 3-4          

In [12]:
dummy_input = torch.randn(16, 3, 21600).to(device)  # Example size, adjust as needed

# Forward pass to get the output
model_output = model(dummy_input)

# Generate the visualization
graph = make_dot(model_output, params=dict(model.named_parameters()))

# Save the visualization to a file
graph.render(r'C:\Users\user\Documents\Python Scripts', format='png')

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\user\\Documents\\Python Scripts'